# Lesson 10: Writer and Image Agents

In the last lesson, we built the Research Agent and Outline Agent. Now we'll build the final 2 agents:

3. **Writer Agent** — Writes the full article from the outline
4. **Image Agent** — Finds and inserts images (optional)

```
ContentOutline --> [Writer Agent] --> Markdown article --> [Image Agent] --> Enriched article
```

Key difference: the Writer Agent uses **Grok-4** (xAI), not Claude. Different models have different strengths — we pick the best model for each task.

## First: What is Markdown?

The Writer Agent produces articles in **Markdown** — a simple text format for creating formatted documents. All files in the `content/` folder are Markdown (`.md`) files.

Here's how Markdown works:

| What you type | What it looks like |
|---|---|
| `# Title` | Big heading (H1) |
| `## Section` | Medium heading (H2) |
| `### Subsection` | Small heading (H3) |
| `**bold text**` | **bold text** |
| `- item` | Bullet point |
| `1. item` | Numbered list |
| `[link text](url)` | Clickable link |
| `![alt text](image-url)` | An image |

**Why Markdown?**
- It's plain text — easy to store, version, and process in code
- It converts to HTML for websites with one command
- It's the standard format for content management systems
- You're reading Markdown right now — this notebook uses it for the text cells!

## Why Grok for the Writer?

In our product, we choose different models for different agents:

| Agent | Model | Reason |
|-------|-------|--------|
| Research | Claude Sonnet | Needs tools (web search) + structured thinking |
| Outline | Claude Sonnet | Needs output_schema (JSON) |
| **Writer** | **Grok-4** | **Natural writing style, flexible tone** |
| Image | Claude Sonnet | Needs tools (API calls) + output_schema |

### Grok's Limitation

Grok has one **important limitation**: it cannot use `tools` and `output_schema` at the same time. So:
- Writer Agent has **no tools** — it only receives text and writes
- Writer Agent has **no output_schema** — it returns plain Markdown directly

Claude supports both together, which is why Research and Outline Agents could potentially be combined into one agent (but we keep them separate for clarity and maintainability).

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from agno.agent import Agent
from agno.models.xai import xAI

writer_agent = Agent(
    name="Writer Agent",
    model=xAI(id="grok-4"),
    instructions=[
        "You are an expert SEO content writer.",
        "Write a comprehensive, SEO-optimized article following the provided outline exactly.",
        "Use proper Markdown: H1 for title, H2 for sections, H3 for subheadings.",
        "Bold important keywords naturally.",
        "Aim for 1500-2500 words.",
        "Do NOT output anything other than the article Markdown.",
    ],
    markdown=True,
)

## Test the Writer Agent

We'll pass a **sample outline** (as a JSON string) to the Writer Agent. In the real product, this outline comes from the Outline Agent in the previous step.

The Writer Agent will:
- Read the outline and understand the structure
- Write a complete article in Markdown
- Ensure every section from the outline is covered
- Naturally weave keywords throughout the article

> **Cost:** ~$0.50-1.00 (Grok-4 writing a long article). Takes 1-2 minutes.

In [ ]:
sample_outline = """{
  "title": "On-Page SEO Guide 2026",
  "meta_description": "Learn how to optimize on-page SEO to boost your website rankings.",
  "target_keywords": ["on-page SEO", "SEO optimization"],
  "sections": [
    {"heading": "What Is On-Page SEO?", "key_points": ["Definition", "Why it matters"]},
    {"heading": "Optimizing Title Tags", "key_points": ["Ideal length", "Keyword placement"]},
    {"heading": "Conclusion", "key_points": ["Summary", "Call to action"]}
  ],
  "tone": "informative"
}"""

print("Writing article from outline...")
response = writer_agent.run(f"Write a full SEO article based on this outline:\n\n{sample_outline}")
article = response.content
print(f"Done! {len(article.split())} words\n")
print(article[:1000] + "\n...")

## Image Agent (Optional)

The last agent in the pipeline. Its job:
- Read the article and identify where images would be helpful
- Search for relevant images via **Freepik API** or **DataForSEO API**
- Insert images into the article with SEO-friendly alt text

### Why is it optional?

The Image Agent needs API keys from Freepik or DataForSEO. If you don't have them:
- The pipeline **skips** this step entirely
- Articles are still created normally, just without images
- No errors occur

This is an intentional design — the product works even without purchasing image API keys.

In [ ]:
import os
from agno.models.anthropic import Claude

# Show the concept (don't actually run without keys)
print("Image Agent is optional:")
print(f"  FREEPIK_API_KEY: {'Set' if os.getenv('FREEPIK_API_KEY') else 'Not set'}")
print(f"  DATA_FOR_SEO_API_KEY: {'Set' if os.getenv('DATA_FOR_SEO_API_KEY') else 'Not set'}")
print()
print("If no API keys are set, the pipeline skips this step.")
print("Articles are still created normally, just without images.")

## Summary

We've now built all **4 agents** for the SEO pipeline:

```
Topic
  |-> [Research Agent]  -- Claude Sonnet + DuckDuckGo    --> Research notes
  |-> [Outline Agent]   -- Claude Sonnet + output_schema --> ContentOutline (JSON)
  |-> [Writer Agent]    -- Grok-4, plain Markdown        --> Full article
  |-> [Image Agent]     -- Claude Sonnet + image APIs    --> Enriched article (optional)
```

**Key takeaways:**
- Choose the right model for each task (Claude for tools/schema, Grok for writing)
- Grok limitation: can't use tools + output_schema together
- Design optional features gracefully — the pipeline works even without image API keys

**Next lesson**: Connect all agents into a complete pipeline with database tracking!

## Exercise

Look at the `article` variable from the writer test above. Write code to:

1. Count the total words (`len(article.split())`)
2. Count how many H2 headings (`##`) are in the article (hint: `article.count("## ")`)
3. Check if the article contains the keyword "on-page SEO" (hint: `"on-page SEO".lower() in article.lower()`)

This is how you'd do basic quality checks on generated content before publishing.

In [ ]:
# Exercise: Write your code here
